In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
import json
import pandas as pd
import numpy as np
from googletrans import Translator
from collections import defaultdict
import requests


In [2]:

# Initialize a list to store the data for each individual page
all_data = {}

# Initialize the web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# Initial search page URL
url = "https://samlingar.shm.se/sok?type=object&productionPeriod=Vikingatid&hasImage=1&category=Arkeologisk%20samling&category=Handel%20och%20v%C3%A4rdem%C3%A4tare&listType=archaeological&rows=300&offset=0"
driver.get(url)

# Wait for the page to load (you may need to adjust the sleep duration)
time.sleep(2)

# Find the cookie disclaimer button by its aria-label
cookie_disclaimer = driver.find_element(By.CSS_SELECTOR, '[aria-label="Godkänn alla kakor"]')
# Check if the cookie disclaimer button is displayed and then click it
if cookie_disclaimer.is_displayed():
    ActionChains(driver).move_to_element(cookie_disclaimer).click().perform()


In [4]:
# Scrape data from the main table
table = driver.find_element(By.TAG_NAME, "table")
df = pd.read_html(table.get_attribute('outerHTML'))[0]

# Drop the 'Bild' column
if 'Bild' in df.columns:
    df.drop(columns=['Bild'], inplace=True)

# Extract museum names from the 'title' attribute of each <i> element
museum_elements = driver.find_elements(By.CSS_SELECTOR, "td i.museum-icon")
museum_names = [elem.get_attribute('title') for elem in museum_elements]

# Replace the 'Museum' column with extracted text
if 'Museum' in df.columns and len(museum_names) == len(df):
    df['Museum'] = museum_names
else:
    print("Mismatch in number of rows while extracting museum names")


In [5]:
# Collect names and href links for each item
item_link = [(item.get_attribute('href')) for item in driver.find_elements(By.CLASS_NAME, "archaeological-list__link")]

item_names = [f"{item.text} - {index+1}" for index, item in enumerate(driver.find_elements(By.CLASS_NAME, "archaeological-list__link"))]
df['Unique Name'] = item_names
df['Catalog Link'] = item_link

# Initialize a column for extra details
df['Extra Details'] = None


In [7]:
# Navigate to each item's link and scrape additional details
for index, link in enumerate(df['Catalog Link']):
    driver.get(link)

    # Scrape the details from the item's page
    item_details = {}
    item_tables = driver.find_elements(By.TAG_NAME, "table")
    for item_table in item_tables:
        item_df = pd.read_html(item_table.get_attribute('outerHTML'))[0]
        for row in item_df.itertuples(index=False):
            item_details[row[0]] = row[1]

    # Store the scraped details as JSON in the DataFrame
    df.at[index, 'Extra Details'] = json.dumps(item_details)


In [ ]:
# Close the driver
driver.quit()

# Replace '-' with NaNs in the entire DataFrame
df.replace('-', np.NaN, inplace=True)

df.head()  # Display the first few rows of the table


,Föremålsbenämning,Föremålsnr.,Förvärvsnr.,Andra nummer,Material,Plats,Fornlämning,Socken,Landskap,Land,Kontexttyp,Kontextnr.,Artbedömning,Benslagsbedömning,Museum,Unique Name,Catalog Link,Extra Details
0,Barr,795_HST,33759,Undernummer: 30,Silver,Spillings,L1976:7626,Othem socken,Gotland,Sverige,Skattfynd,2,NaN,NaN,Historiska museet,Barr - 1,https://samlingar.shm.se/object/4CCBEB01-351D-...,"{""Museum"": ""Historiska museet"", ""F\u00f6rem\u0..."
1,Barr,897_HST,33759,Undernummer: 211,Silver,Spillings,L1976:7626,Othem socken,Gotland,Sverige,Skattfynd,2,NaN,NaN,Historiska museet,Barr - 2,https://samlingar.shm.se/object/4C648ABC-1BFC-...,"{""Museum"": ""Historiska museet"", ""F\u00f6rem\u0..."
2,spiralring,911_HST,33759,Undernummer: 232,Silver,Spillings,L1976:7626,Othem socken,Gotland,Sverige,Skattfynd,2,NaN,NaN,Historiska museet,spiralring - 3,https://samlingar.shm.se/object/9345E7C4-0C1B-...,"{""Museum"": ""Historiska museet"", ""F\u00f6rem\u0..."
3,Konglomerat,947_HST,33759,Undernummer: 283,Silver,Spillings,L1976:7626,Othem socken,Gotland,Sverige,Skattfynd,2,NaN,NaN,Historiska museet,Konglomerat - 4,https://samlingar.shm.se/object/870C041D-7B18-...,"{""Museum"": ""Historiska museet"", ""F\u00f6rem\u0..."
4,Barr,1006_HST,33759,Undernummer: 357,Silver,Spillings,L1976:7626,Othem socken,Gotland,Sverige,Skattfynd,2,NaN,NaN,Historiska museet,Barr - 5,https://samlingar.shm.se/object/67531AE6-EBF5-...,"{""Museum"": ""Historiska museet"", ""F\u00f6rem\u0..."


In [ ]:
# These are my own functions that help clean the data

def search_columns_for_substring(columns, substring):
    return [col for col in columns if substring in col]

# this function counts how many empty values are in each column for a data set (use this after the drop col function to verify)
def count_nans_in_dataframe(df):
    nan_counts = df.isna().sum()
    return pd.DataFrame({'Column': nan_counts.index, 'NaN Count': nan_counts.values})

# this function drops columns that do not meet the minimum threshold
# if by="count" then drop columns that don't have at least that many populated fields
    # ie. by_val=100 drops columns that have less than 100 populated rows
# if by="prop" then drop columns that don't have at least by_val% populated rows
    # ie. by_val=0.05 drops columns that aren't at least 5% populated
# if by="field" then drop columns that have more missing values than the columns specified
    # ie. by_val="Last_Device_Array.anv" will keep Last_Device_Array.anv but drop any cols that have more missing values than Last_Device_Array.anv
def drop_columns_with_fewer_nans(df, by="prop", by_val="0.05"):
    if by == "count":
        threshold = float(by_val)
    elif by == "prop": threshold = round(df.shape[0]*float(by_val))
    elif by == "field": threshold = df.shape[0]-df[by_val].isna().sum()
    cols_to_drop = []
    for col in df.columns:
        if df[col].isna().sum() > df.shape[0]-threshold:
            cols_to_drop.append(col)
    df = df.drop(cols_to_drop, axis=1)
    return df

# this function to drops columns with duplicate names
def drop_duplicate_columns(df):
    duplicates = df.columns[df.columns.duplicated(keep='first')]
    df = df.loc[:, ~df.columns.duplicated(keep='first')]
    return df

In [ ]:
###### UNPACK JSON: TRADE ARTIFACTS
# Load the dataset
# dataset = pd.read_csv('/Users/aly.milne/Library/CloudStorage/OneDrive-BrighamYoungUniversity/Fall 2023/STAT 386/ST386_Final_Project/Scraped_Data/Viking_trade_artifacts.csv')

# unpack json
df['Extra Details'] = df['Extra Details'].map(json.loads)
trade_artifacts_exploded = pd.json_normalize(df.to_dict(orient='records'))

# drop rows that are not at least 25% populated
cleaned_trade_artifacts = drop_columns_with_fewer_nans(trade_artifacts_exploded, "prop", 0.25)

# Removing 'Extra Details.' prefix from column names
cleaned_trade_artifacts.columns = cleaned_trade_artifacts.columns.str.replace('Extra Details.', '', regex=False)

# Applying the function to your dataframe
cleaned_trade_artifacts = drop_duplicate_columns(cleaned_trade_artifacts)

In [ ]:
# Initialize the translator and cache
translator = Translator()
translations_cache = defaultdict(str)

# Function to batch translate a list of texts
def batch_translate(texts, src='sv', dest='en'):
    # Filter out None values and ensure text is string
    filtered_texts = [str(text) for text in texts if pd.notna(text)]
    
    # Batch processing and caching
    batch_size = 10  # Adjust batch size as needed
    for i in range(0, len(filtered_texts), batch_size):
        batch = filtered_texts[i:i+batch_size]
        untranslated_batch = [text for text in batch if text not in translations_cache]

        if untranslated_batch:
            try:
                translations = translator.translate(untranslated_batch, src=src, dest=dest)
                for text, translation in zip(untranslated_batch, translations):
                    translations_cache[text] = translation.text
            except Exception as e:  # General exception catch
                print(f"Error during translation: {e}")
                for text in untranslated_batch:
                    translations_cache[text] = text

        time.sleep(0.5)  # Respect rate limits

    return [translations_cache[text] for text in texts]


In [ ]:

###### TRANSLATE
# Load your datasets
df_trade_translate = cleaned_trade_artifacts

# Identify text columns in the dataframes
text_columns_trade = df_trade_translate.select_dtypes(include=['object']).columns

# Translate text columns
for df, text_columns in [(df_trade_translate, text_columns_trade)]:
    for column in text_columns:
        print(f"Translating column: {column}")
        df[column + '_Translated'] = batch_translate(df[column].tolist())

df_trade_translate.head(5)


Translating column: Föremålsbenämning
Error during translation: the JSON object must be str, bytes or bytearray, not NoneType
Error during translation: the JSON object must be str, bytes or bytearray, not NoneType
Error during translation: the JSON object must be str, bytes or bytearray, not NoneType
Error during translation: the JSON object must be str, bytes or bytearray, not NoneType
Error during translation: the JSON object must be str, bytes or bytearray, not NoneType


KeyboardInterrupt: 

In [ ]:
def geocode(location):
    """ Geocode a location using Nominatim API. """
    url = 'https://nominatim.openstreetmap.org/search'
    headers = {
        'User-Agent': 'alyanngirl@gmail.com'
    }
    params = {
        'q': location,
        'format': 'json'
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        results = response.json()
        if results:
            return results[0]['lat'], results[0]['lon']
    return None, None


In [ ]:
###### GET LOCATIONS: TRADE ARTIFACTS
# Add columns for latitude and longitude
cleaned_trade_artifacts['latitude'] = None
cleaned_trade_artifacts['longitude'] = None

# Iterate over the DataFrame and geocode each location
for index, row in cleaned_trade_artifacts.iterrows():
    location = row['Plats']
    lat, lon = geocode(location)
    cleaned_trade_artifacts.at[index, 'latitude'] = lat
    cleaned_trade_artifacts.at[index, 'longitude'] = lon

    # Respect Nominatim's usage policy
    time.sleep(1)

# Save the updated DataFrame
cleaned_trade_artifacts.to_csv('ST386_Final_Project/Scraped_Data/Viking_trade_artifacts.csv', index=False)

In [ ]:
# alternates: 
# https://samlingar.shm.se/sok?type=object&productionPeriod=Vikingatid&hasImage=1&category=Arkeologisk%20samling&category=Vapen%20och%20rustningar&listType=archaeological&rows=500&offset=0
# https://samlingar.shm.se/sok?type=object&productionPeriod=Vikingatid&hasImage=1&category=Arkeologisk%20samling&category=Handel%20och%20v%C3%A4rdem%C3%A4tare&listType=archaeological&rows=300&offset=0
# https://samlingar.shm.se/sok?type=object&query=Vikingatid&listType=archaeological&rows=1000&offset=0

In [ ]:
###### TRANSLATE: TRADE ARTIFACTS
# Columns to translate
columns_to_translate = cleaned_trade_artifacts.columns.to_list()

translated_trade = []

# Translate the columns
for column in columns_to_translate:
    cleaned_trade_artifacts[column] = cleaned_trade_artifacts[column].apply(lambda x: translate_text(x) if isinstance(x, str) else x)

cleaned_trade_artifacts.head(5)

# Save the translated dataframe to a new CSV file
# cleaned_trade_artifacts.to_csv('translated_viking_trade_artifacts.csv', index=False)

In [ ]:
###### TRANSLATE: TRADE ARTIFACTS
# Split the DataFrame into smaller chunks if it's very large
chunk_size = 100  # Adjust the chunk size based on your data and rate limits
chunks = [cleaned_trade_artifacts[i:i + chunk_size] for i in range(0, cleaned_trade_artifacts.shape[0], chunk_size)]

# Process each chunk
for chunk in chunks:
    chunk['Translated_Description'] = batch_translate(chunk['Description'].tolist())
    time.sleep(1)  # Respect rate limits

# Concatenate chunks back into a single DataFrame
translated_trade_df = pd.concat(chunks)
translated_trade_df.head(5)

# Save the translated DataFrame to a new file
# translated_df.to_csv('/path/to/your/translated_dataset.csv', index=False)